1.使用TensorFlow实现IOU计算

In [1]:
import tensorflow as tf
def calc_iou(box1,box2):
    left_up = tf.maximum(box1[:2],box2[:2])
    right_down = tf.minimum(box1[2:],box2[2:])
    intersection = tf.maximum(0.,right_down - left_up)
    inter_square = intersection[0] * intersection[1]
    
    box1_square = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_square = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    iou = inter_square / (box1_square + box2_square - inter_square + 1e-8)
    
    return iou

In [2]:
with tf.Session() as sess:
    iou = calc_iou([10.,20.,60.,60.],[50.,40.,80.,90.])
    print(sess.run(iou))
    iou = calc_iou([10.,20.,40.,60.],[50.,40.,80.,90.])
    print(sess.run(iou))

0.060606062
0.0


2.计算AP，mAP

In [3]:
import sklearn.metrics as metrics
metrics.average_precision_score([0,0,1,1,1],[0.2,0.4,0.5,0.52,0.9])

1.0

In [4]:
AP1 = metrics.average_precision_score([0,0,1],[0.2,0.1,0.7])
AP2 = metrics.average_precision_score([1,0,0],[0.4,0.5,0.1])
AP3 = metrics.average_precision_score([0,0,1],[0.1,0.15,0.75])
AP4 = metrics.average_precision_score([1,0,0],[0.3,0.4,0.3])
AP5 = metrics.average_precision_score([0,1,0],[0.6,0.3,0.1])
AP6 = metrics.average_precision_score([0,1,0],[0.25,0.65,0.1])
AP7 = metrics.average_precision_score([1,0,0],[0.9,0.05,0.05])
AP8 = metrics.average_precision_score([0,1,0],[0.35,0.45,0.2])
mAP = (AP1+AP2+AP3+AP4+AP5+AP6+AP7+AP8)/8
mAP

0.7916666666666667

3.NMS算法实现

In [5]:
#1.使用Numpy实现IOU计算
import numpy as np
def calc_iou(box1,box2):
    left_up = np.maximum(box1[:2],box2[:2])
    right_down = np.minimum(box1[2:],box2[2:])
    intersection = np.maximum(0.,right_down - left_up)
    inter_square = intersection[0] * intersection[1]
    
    box1_square = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_square = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    iou = inter_square / (box1_square + box2_square - inter_square + 1e-8)
    
    return iou

In [6]:
#2.实现单一类别的NMS计算
def nms(boxes,threshold=0.7):
    boxes.sort(key=lambda boxes:boxes[4],reverse=True)
    res_boxes = []
    while(len(boxes) != 0):
        res_boxes.append(boxes[0])
        del boxes[0]
        
        keep_num = 0
        for i in range(len(boxes)):
            iou = calc_iou(res_boxes[-1][0:4],boxes[keep_num][0:4])
            if iou > threshold:
                del boxes[keep_num]
            else:
                keep_num += 1
    return res_boxes

In [7]:
#3.测试NMS算法
boxes = [[0,0,50,50,0.9],
         [0,0,60,50,0.85],
         [40,40,70,70,0.4],
         [35,45,70,80,0.7]]
res_boxes = nms(boxes,0.5)
print(res_boxes)

[[0, 0, 50, 50, 0.9], [35, 45, 70, 80, 0.7]]


4.TensorFlow实现代价函数

In [8]:
#模拟模型输出与数据标记
outputs = tf.random_uniform([2,8],0,1,dtype=tf.float32)
labels = tf.constant([[0,0,0,0,0,0,0,0],[1,0.2,0.3,0.3,0.8,0,0,1]],dtype=tf.float32)
label_confidence = labels[:,0:1]
#根据label_confidence生成掩码
mask = tf.concat([tf.ones_like(labels)[:,0:1],
                  (tf.ones_like(labels)*label_confidence)[:,1:]],1)
#对模型输出进行过滤，使label_confidence==0的模型输出除了置信度以外的值设为0
outputs = outputs * mask

#使用均方误差代价函数表示代价
cost = tf.reduce_mean(tf.square(labels - outputs))